This will be focused on generating the voice locally.
- the script will be passed on (or queried in the vector db from the latest entry )
- then call the TTS api to actually convert it into an mp3

In [1]:
from TTS.api import TTS
import soundfile as sf
import librosa

In [2]:
from langchain_community.llms import Ollama # will be used for prompting
from langchain.vectorstores import Chroma # will be used for vectordb store
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document # will be used to store text in vector store 
import os

In [3]:
llm = Ollama(model="llama3")
embedding = OllamaEmbeddings(model="llama3")
PERSIST_DIR = "vectordb"
db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_22453/1332557208.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")
/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_22453/1332557208.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)


In [4]:
from langchain.schema import Document  # or adjust import to your LangChain version

def get_most_recent_unprocessed_idea(db):
    results = db.get(include=["documents", "metadatas"])

    unprocessed = [
        (meta["timestamp"], Document(page_content=doc, metadata=meta))
        for meta, doc in zip(results["metadatas"], results["documents"])
        if not meta.get("processed", False)
    ]

    if not unprocessed:
        print("No unprocessed ideas found.")
        return None

    unprocessed_sorted = sorted(unprocessed, key=lambda x: x[0], reverse=True)
    _, document = unprocessed_sorted[0]

    print(f"Most recent unprocessed idea ID: {document.metadata.get('id')}")
    return document


In [5]:
most_recent_idea = get_most_recent_unprocessed_idea(db)
if most_recent_idea:
    print(most_recent_idea.page_content) 


Most recent unprocessed idea ID: 85c9b47a-0d2f-4f36-b3f9-8a22087c9489
Hey everyone, have you ever wondered how some of your favorite e-commerce sites can handle millions of orders in a single day without breaking a sweat? Well, it's all thanks to Amazon Simple Queue Service, or SQS for short.

So, what is SQS? Simply put, it's a messaging service that allows different parts of an application to communicate with each other without actually being connected. Think of it like a postal system - your app sends a message to the queue, and then another part of the app can pick it up whenever it's ready.

You'd use SQS when you have multiple microservices or tasks that need to be processed in a specific order, but they don't necessarily need to talk to each other directly. For example, imagine you're building an online ordering system where orders are processed by different teams - customer support, fulfillment, and shipping. Each team needs to receive the order at a different time, and SQS hel

In [6]:
tts = TTS(model_name="tts_models/en/ljspeech/glow-tts", progress_bar=False, gpu=False)

# Generate audio
tts.tts_to_file(
    text=most_recent_idea.page_content,
    file_path="output_raw.wav"
)

# Speed up to 1.5x
y, sr = librosa.load("output_raw.wav", sr=None)
y_fast = librosa.effects.time_stretch(y, rate=1.5)
sf.write("output.wav", y_fast, sr)

 > tts_models/en/ljspeech/glow-tts is already downloaded.
 > vocoder_models/en/ljspeech/multiband-melgan is already downloaded.
 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Vocoder Model: multiband_melgan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resam

In [ ]:
import stable_whisper

def generate_srt_with_stable_whisper(audio_path="output.wav", model_size="tiny"):
    model = stable_whisper.load_model(model_size)  # Options: tiny, base, small, medium, large
    result = model.transcribe(audio_path)

    # Save SRT file
    result.save_as_srt("output.srt")
    print("✅ Subtitles saved as output.srt")

# Run it
generate_srt_with_stable_whisper()


ModuleNotFoundError: No module named 'subtitles_generator'